In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lukeyang01/nn-from-scratch/main/week2_data.csv')

inputs = np.array(df.iloc[:, 0:2])
labels = np.array(df.iloc[:, 2])

In [ ]:
def sigmoid(x: int) -> float:
  """Sigmoid activation function."""
  return 1 / (1 + np.exp(-x))

def sigmoid_back(x: int) -> float:
  """Derivative of sigmoid for backward calculation."""
  fwd = sigmoid(x)
  return fwd * (1-fwd)

In [ ]:
class MLP:
  """A Neural Network Class to Perform Basic Feedforward algorithm and training"""
  def __init__(self, sizes: list):
    """Initialize a numpy array or a list of weights an array or list of weights depending on sizes"""
    self.sizes = sizes
    self.num_layers = len(sizes)
    self.weights = []
    self.biases = []

    self.__init_params()
    return

  def __init_params(self):
    """Initialize random weights and biases based on size parameters."""
    for i in range(1, self.num_layers):
        # X inputs -> Y Ouputs
        in_size = self.sizes[i-1]
        out_size = self.sizes[i]

        # weights.shape = (Y, X), biases.shape = (Y, 1)
        # e.g. 2 -> 3: weights is (2, 3), biases is (1, 3)
        self.weights.append(np.random.randn(out_size, in_size) * 0.1)
        self.biases.append(np.random.randn(out_size, 1) * 0.1)

  def forward(self, x: np.ndarray):
    """
      Perform feedforward algorithm on input vector for all layers

      Input:    x: np.ndarray with shape (1, self.sizes[0])

      Returns:  y: np.ndarray with shape (1, self.sizes[-1])
    """
    # x = x.reshape(self.sizes[0], 1)

    zs = []
    activations = [x]

    # Store inner layers for backward calculation (?)
    for i in range(self.num_layers-1):
      x = np.matmul(self.weights[i], x) + self.biases[i]
      zs.append(x)
      x = sigmoid(x)
      activations.append(x)


    return x, activations, zs

  def backward(self, x, y):
    """Perform backpropagation using the input and expected output to get weight and vector deltas"""

    delta_b = [np.zeros(b.shape) for b in self.biases]
    delta_w = [np.zeros(w.shape) for w in self.weights]

    output, activations, zs = self.forward(x)
    delta = (output - y) *  sigmoid_back(zs[-1])

    # Back Prop on output layer
    delta_w[-1] = np.matmul(delta, activations[-2].T)
    delta_b[-1] = delta

    for idx in range(2, self.num_layers):
      z = zs[-idx]
      sp = sigmoid_back(z)
      delta = np.matmul(self.weights[-idx + 1].T, delta) * sp
      delta_b[-idx] = delta
      delta_w[-idx] = np.matmul(delta, activations[-idx - 1].T)

    return delta_w, delta_b

  def train(self, X_train, y_train, epochs=1, lr=0.01, batch_size=1, verbose=True):
    """Using forward and backward functions, fit the model on an entire training step using gradient descent algorithm."""
    for x in X_train:
      delta_w, delta_b = self.backward(x.reshape((2, 1)), np.array([0.5, 0.5]).reshape((2, 1)))
      for i in range(len(self.weights)):
        self.weights[i] -= delta_w[i]
        self.biases[i] -= delta_b[i]
    print(self.forward(inputs[10].reshape((2,1)))[0])

In [ ]:
def main():
  nn = MLP([2, 3, 2])
  nn.train(inputs, labels)

In [ ]:
main()

[[0.50432868]
 [0.50802512]]
